In [ ]:
!pip install gdown torch torchtext torchdata stanza spacy --quiet
!python3 -m spacy download en_core_web_sm

In [36]:
import stanza
import spacy
# Prepare Stanford's Stanza to tokenize the columns and tables names
# stanza.download('en')
# stanza_nlp = stanza.Pipeline(processors='tokenize,mwt,pos,lemma', lang='en')

# Prepare spacy to tokenize and lemmatize the table names and columns names
spacy_nlp = spacy.load("en_core_web_sm", disable=["ner", "textcat", "parser"])
# Fix the "Spacy breaks up 'id' into 'i' and 'd' issue" https://github.com/explosion/spaCy/discussions/10570
spacy_nlp.tokenizer.rules = { key: value for key, value in spacy_nlp.tokenizer.rules.items() if key != "id" }

DEVICE = "mps" # Apple Metal Shader (M1 chip)

In [37]:
%%capture
import gdown
import os

spider_url = 'https://drive.google.com/u/0/uc?id=1iRDVHLr4mX2wQKSgA9J8Pire73Jahh0m&export=download' # https://yale-lily.github.io/spider
output = 'spider.zip'
if not os.path.exists(output):
  gdown.download(spider_url, output, quiet=False)

!if [ ! -d "spider" ]; then unzip spider.zip -d .; else echo "The 'spider' folder already exists."; fi

In [38]:
import torch
from torch import nn
import json
from dataclasses import dataclass, field
from typing import Literal, Any, Union, List

# Start by loading in the databases, tables, and columns schemas

with open("spider/tables.json") as f:
  raw_schema_data = json.load(f)

ColumnType = Literal["time", "text", "number", "boolean", "others"]


@dataclass
class Column:
  db_id: str  # Which database this col belongs to
  table_index: int  # Which specific table in the db this col belongs to
  col_index: int  # Which specific column in this table
  name: str
  name_toks: list[str]  # Tokenized name
  orig_name: str
  col_type: ColumnType


@dataclass
class Table:
  db_id: str  # Which database this table belongs to
  table_index: int  # Which specific table in this database
  name: str
  name_toks: list[str]  # Tokenized name
  orig_name: str
  columns: list[Column] = field(default_factory=list)


@dataclass
class Database:
  db_id: str
  raw_schema: Any  # The original raw schema from json file
  tables: list[Table] = field(default_factory=list)
  columns: list[Column] = field(default_factory=list)


schema_lookup: dict[str, Database] = {}  # Map the db_id to that database's schema

raw_schema_data[0].keys()
# ['column_names', 'column_names_original', 'column_types', 'db_id',
# 'foreign_keys', 'primary_keys', 'table_names', 'table_names_original']


def lemmatized_tokenizer(s: str):
  doc = spacy_nlp(s)
  lemmatized_tokens = [token.lemma_.lower() for token in doc]
  # doc = stanza_nlp(s)
  # lemmatized_tokens = [word.lemma.lower() for sent in doc.sentences  for word in sent.words]
  return lemmatized_tokens


for i, db_info in enumerate(raw_schema_data):
  db_id = db_info["db_id"]
  db = Database(db_id, raw_schema=db_info)

  # Iterate the tables of this database
  for table_index, (tbl_name, tbl_orig_name) in enumerate(
    zip(db_info["table_names"], db_info["table_names_original"])
  ):
    new_table = Table(
      db_id,
      table_index=table_index,
      name=tbl_name,
      name_toks=lemmatized_tokenizer(tbl_name),
      orig_name=tbl_orig_name,
    )
    db.tables.append(new_table)

  # Iterate the columns of this database
  for col_index, ((table_idx, col_name), (_, col_orig_name), col_type) in enumerate(
    zip(
        db_info["column_names"],
        db_info["column_names_original"],
        db_info["column_types"],
    )
  ):
    new_col = Column(
        db_id=db_id,
        col_index=col_index,
        col_type=col_type,
        table_index=table_idx,
        name=col_name,
        name_toks=lemmatized_tokenizer(col_name),
        orig_name=col_orig_name,
    )
    db.columns.append(new_col)
    table_idx: int = table_idx
    if table_idx >= 0:  # Skip the -1 index table
        db.tables[table_idx].columns.append(new_col)

  schema_lookup[db_id] = db

all_columns = [col for s in schema_lookup.values() for col in s.columns]
all_tables = [tbl for s in schema_lookup.values() for tbl in s.tables]

print(all_columns[1])
print(schema_lookup["entrepreneur"])


Column(db_id='perpetrator', table_index=0, col_index=1, name='perpetrator id', name_toks=['perpetrator', 'id'], orig_name='Perpetrator_ID', col_type='number')
Database(db_id='entrepreneur', raw_schema={'column_names': [[-1, '*'], [0, 'entrepreneur id'], [0, 'people id'], [0, 'company'], [0, 'money requested'], [0, 'investor'], [1, 'people id'], [1, 'name'], [1, 'height'], [1, 'weight'], [1, 'date of birth']], 'column_names_original': [[-1, '*'], [0, 'Entrepreneur_ID'], [0, 'People_ID'], [0, 'Company'], [0, 'Money_Requested'], [0, 'Investor'], [1, 'People_ID'], [1, 'Name'], [1, 'Height'], [1, 'Weight'], [1, 'Date_of_Birth']], 'column_types': ['text', 'number', 'number', 'text', 'number', 'text', 'number', 'text', 'number', 'number', 'text'], 'db_id': 'entrepreneur', 'foreign_keys': [[2, 6]], 'primary_keys': [1, 6], 'table_names': ['entrepreneur', 'people'], 'table_names_original': ['entrepreneur', 'people']}, tables=[Table(db_id='entrepreneur', table_index=0, name='entrepreneur', name_t

In [141]:
# Load the questions
with open("spider/train_spider.json") as f:
  raw_train_data = json.load(f)

with open("spider/dev.json") as f:
  raw_val_data = json.load(f)

raw_train_data[0].keys()
# ['db_id', 'query', 'query_toks', 'query_toks_no_value', 'question', 'question_toks', 'sql']

@dataclass
class QAPair: # Class to encapsulate each NL query with corresponding SQL answer
  db_id: str
  question_toks: list[str]
  question: str
  query_toks: list[str]
  query: str
  sql_tree: Any # Raw sql tree as a nested dict from the raw JSON

@dataclass
class SpiderItem:
  """An item to feed into the neural network. Basically encapsulation of a training example"""
  db_id: str
  qa_pair: QAPair
  db: Database # Information about the database used in the qa


train_qas: list[QAPair] = []
val_qas: list[QAPair] = []
train_items: list[SpiderItem] = []
val_items: list[SpiderItem] = []

for i, qa_info in enumerate(raw_train_data):
  db_id = qa_info["db_id"]
  new_qa = QAPair(
    db_id=db_id,
    question_toks=qa_info["question_toks"],
    question=qa_info["question"],
    query_toks=qa_info["query_toks"],
    query=qa_info["query"],
    sql_tree=qa_info["sql"]
  )
  new_item = SpiderItem(
    db_id=db_id,
    qa_pair=new_qa,
    db=schema_lookup[db_id]
  )
  train_qas.append(new_qa)
  train_items.append(new_item)

for i, qa_info in enumerate(raw_val_data):
  db_id = qa_info["db_id"]
  new_qa = QAPair(
    db_id=db_id,
    question_toks=qa_info["question_toks"],
    question=qa_info["question"],
    query_toks=qa_info["query_toks"],
    query=qa_info["query"],
    sql_tree=qa_info["sql"]
  )
  new_item = SpiderItem(
    db_id=db_id,
    qa_pair=new_qa,
    db=schema_lookup[db_id]
  )
  val_qas.append(new_qa)
  val_items.append(new_item)


In [40]:
col_name_vocab: set[str] = set()
tbl_name_vocab: set[str] = set()
col_types: set[str] = set()

# Build col names and tbl names vocab
for col in all_columns:
  col_name_vocab.update(col.name_toks)
  col_types.add(col.col_type)

for tbl in all_tables:
  tbl_name_vocab.update(tbl.name_toks)

print(col_types) # {'text', 'others', 'number', 'time', 'boolean'}
print("Vocab size col names", len(col_name_vocab))
print("Vocab size tbl names", len(tbl_name_vocab))
print(list(col_name_vocab)[0:10])
print(list(tbl_name_vocab)[0:10])

{'boolean', 'time', 'text', 'number', 'others'}
Vocab size col names 1176
Vocab size tbl names 506
['regular', 'performance', 'dimension', 'uid', 'year', '600', 'uk', 'mark', 'student', 'key']
['regular', 'performance', 'key', 'student', 'chip', 'breed', 'author', 'show', 'channel', 'culture']


In [139]:
target_vocab: set[str] = set() # Vocab of the SQL target
question_vocab: set[str] = set() # Vocab of the Natural Language questions

for qa_pair in train_qas:
  question_vocab.update(qa_pair.question_toks)
  target_vocab.update(qa_pair.query_toks)

for qa_pair in val_qas:
  question_vocab.update(qa_pair.question_toks)
  target_vocab.update(qa_pair.query_toks)

# Padding token used to pad shorter inputs so every sequence of inputs to the encoder
# is of same length (which would be the max length)
PAD_TOKEN = "<PAD>"

src_vocab: set[str] = set() # Source vocab used for inputs to the encoder, which includes questions & columns & tables names
src_vocab.update(col_name_vocab)
src_vocab.update(col_types)
src_vocab.update(tbl_name_vocab)
src_vocab.update(question_vocab)
# Add the "CLS" token as a separator between each part in the input block (i.e. separator between each column name, table name, etc.)
src_vocab.add("CLS")
src_vocab.add(PAD_TOKEN)

tgt_vocab: set[str] = set() # Target vocab used for decoder output; which is the set of SQL tokens plus the Start and End tokens
tgt_vocab.update(target_vocab)
tgt_vocab.add("<START>")
tgt_vocab.add("<END>")
tgt_vocab.add(PAD_TOKEN)

print("Vocab size questions", len(question_vocab))
print("Vocab size SQL", len(target_vocab))

# Token to id: Assign a number to each token in the vocab
# Id to token: Retrieve the token give the index
src_tok_to_idx = { tok: i for i, tok in enumerate(src_vocab) }
src_idx_to_tok = { i: tok for i, tok in enumerate(src_vocab) }
tgt_tok_to_idx = { tok: i for i, tok in enumerate(tgt_vocab) }
tgt_idx_to_tok = { i: tok for i, tok in enumerate(tgt_vocab) }

def toks_encode(tokens_list: list[str], mode: Literal["source", "target"]):
  """Given a list of tokens, 'encode' them to return a list of token_ids
  Parameters:
    mode: Whether to use the source's or target's tok_to_idx lookup table
  """
  if mode == "source":
    return [src_tok_to_idx[token] for token in tokens_list]
  return [tgt_tok_to_idx[token] for token in tokens_list]

def toks_decode(ids_list: list[int], mode: Literal["source", "target"]):
  """Given a list of token_ids, 'decode' them to return the list of corresponding token
  Parameters:
    mode: Whether to use the source's or target's idx_to_tok lookup table
  """
  if mode == "source":
    return [src_idx_to_tok[token_id] for token_id in ids_list]
  return [tgt_idx_to_tok[token_id] for token_id in ids_list]


Vocab size questions 3750
Vocab size SQL 5468


In [140]:
# Create the dataset and dataloader
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence

def construct_input_block_toks(
    col_toks: list[list[str]],
    col_types: list[ColumnType],
    table_toks: list[list[str]],
    ques_toks: list[str],
):
    """Construct and return the 3 components of an input block as a tuple of the list of tokens
    An input block has 3 parts:
      The col name part: `CLS col_type ...col_name CLS col_type2 ...Col_name2`
      The table name part: `CLS ...tbl_name CLS ...tbl_name2`
      The question part: ` CLS ...question`

    Parameters:
      col_toks: nested list of all tokens across all column names, each element is a list of tokens that belongs in one column name
      tbl_toks: nested list of all tokens across all table names, each element is a list of tokens that belongs in one table name

    Returns:
      a 3-elemment tuple containing (col_name_part, tbl_name_part, ques_part) w/ each part being the list of tokens including the 'CLS' token
    """
    # List of column types should match length with list of columns
    assert len(col_toks) == len(col_types)
    # Construct the col name part, thus producing something like:
    # "CLS number builing size CLS text building name..."
    col_name_part: list[str] = []
    for col_name_toks, col_type in zip(col_toks, col_types):
        col_name_part.extend(["CLS", col_type] + col_name_toks)
    # Construct the col name part, thus producing something like:
    # "CLS building information CLS city information ...etc"
    tbl_name_part: list[str] = []
    for tbl_name_toks in table_toks:
        tbl_name_part.extend(["CLS"] + tbl_name_toks)
    # Construct the question part, thus producing something like:
    # "CLS what is the highest building in Chicago"
    question_part: list[str] = ques_toks
    return (col_name_part, tbl_name_part, question_part)

DatasetItem = tuple[list[str], list[str], list[str], list[str]]
class MyDataset(Dataset[DatasetItem]):
    """Each element in this dataset is a 4-element tuple
    (col_name_toks, table_name_toks, question_toks, target_toks)

    where `col_name_toks`, `table_name_toks`, `question_toks` has been preprocessed to add "CLS" separator token
    and `target_toks` is also processed to include <Start> and <End> tokens
    """
    def __init__(self, all_items: list[SpiderItem]):
        self.items = all_items

    def __len__(self):
        return len(self.items)

    def __getitem__(self, idx: int) -> DatasetItem:
        item = self.items[idx]
        columns_tokens = [c.name_toks for c in item.db.columns[1:]] # Skip the 0th-index "*" column
        columns_types: list[ColumnType] = [c.col_type for c in item.db.columns[1:]]
        tables_tokens = [t.name_toks for t in item.db.tables]
        question_tokens = item.qa_pair.question_toks
        target_tokens = ["<START>"] + item.qa_pair.query_toks + ["<END>"]

        return construct_input_block_toks(columns_tokens, columns_types, tables_tokens, question_tokens) + (target_tokens,)


train_dataset = MyDataset(train_items)
val_dataset = MyDataset(val_items)
print(len(train_dataset))  # Should be 7000
print(len(val_dataset)) # Should be 1034

PAD_TOKEN_ID = src_tok_to_idx[PAD_TOKEN]

# Find the max_length of each "category".
# Category here correspondings to the 4 categories (col_name_toks, table_name_toks, question_toks, target_toks)
max_lengths = [max(len(category) for category in item) for item in zip(*train_dataset)]
print(max_lengths) # [1196, 70, 44, 92]
max_len_col_name_part = max_lengths[0]
max_len_tbl_name_part = max_lengths[1]
max_len_ques_part = max_lengths[2]
max_len_target = max_lengths[3]

BATCH_SIZE = 64

def pad_collate(batch: list[tuple[list[str], list[str], list[str], list[str]]]):
    """The collate_fn to pass to Pytorch Dataloader.
    Pad each element in the dataset so they all have the same size"""
    batched_col_name_toks, batched_tbl_name_toks, batched_ques_toks, batched_target_toks = zip(*batch)

    batched_colname_tokIds = [torch.tensor(toks_encode(col_name_toks, mode="source"), dtype=torch.long) for col_name_toks in batched_col_name_toks]
    # Now pad this batch so that every item of the batch is same size as a local longest item
    # From the doc: `pad_sequence` stacks a list of Tensors along a new dimension, and pads them to equal length.
    batched_colname_tokIds = pad_sequence(batched_colname_tokIds, batch_first=True, padding_value=PAD_TOKEN_ID)
    # Now pad this batch again so that every item of the batch is same size of the global longest item (max_len_col_name_part)
    batched_colname_tokIds= torch.nn.functional.pad(
        batched_colname_tokIds, (0, max_len_col_name_part - batched_colname_tokIds.size(1)), value=PAD_TOKEN_ID)
    batched_colname_tokIds = batched_colname_tokIds.to(DEVICE) # (B, T)

    # After padding, each tensor should be dimension (B, T)
    # With B being the batch dimension (since we use batch_first=True) and T is max_len_col_name_part
    assert batched_colname_tokIds.shape == (BATCH_SIZE, max_len_col_name_part), f"Expected {BATCH_SIZE, max_len_col_name_part}, got {batched_colname_tokIds.shape}"

    # Do the same process for table name toks, question toks, and target toks
    batched_tblname_tokIds = [torch.tensor(toks_encode(tbl_name_toks, mode="source")) for tbl_name_toks in batched_tbl_name_toks]
    batched_tblname_tokIds = pad_sequence(batched_tblname_tokIds, batch_first=True, padding_value=PAD_TOKEN_ID)
    batched_tblname_tokIds= torch.nn.functional.pad(
        batched_tblname_tokIds, (0, max_len_tbl_name_part - batched_tblname_tokIds.size(1)), value=PAD_TOKEN_ID)
    batched_tblname_tokIds = batched_tblname_tokIds.to(DEVICE)
    assert batched_tblname_tokIds.shape == (BATCH_SIZE, max_len_tbl_name_part)

    batched_ques_tokIds = [torch.tensor(toks_encode(ques_toks, mode="source")) for ques_toks in batched_ques_toks]
    batched_ques_tokIds = pad_sequence(batched_ques_tokIds, batch_first=True, padding_value=PAD_TOKEN_ID)
    batched_ques_tokIds = torch.nn.functional.pad(
        batched_ques_tokIds, (0, max_len_ques_part - batched_ques_tokIds.size(1)), value=PAD_TOKEN_ID)
    batched_ques_tokIds = batched_ques_tokIds.to(DEVICE)
    assert batched_ques_tokIds.shape == (BATCH_SIZE, max_len_ques_part)

    batched_tgt_tokIds = [torch.tensor(toks_encode(tgt_toks, mode="target")) for tgt_toks in batched_target_toks]
    # Note that for the target, we need use `tgt_tok_to_idx`
    batched_tgt_tokIds = pad_sequence(batched_tgt_tokIds, batch_first=True, padding_value=tgt_tok_to_idx[PAD_TOKEN])
    batched_tgt_tokIds = torch.nn.functional.pad(
        batched_tgt_tokIds, (0, max_len_target - batched_tgt_tokIds.size(1)), value=PAD_TOKEN_ID)
    batched_tgt_tokIds = batched_tgt_tokIds.to(DEVICE)
    assert batched_tgt_tokIds.shape == (BATCH_SIZE, max_len_target)

    return (batched_colname_tokIds, batched_tblname_tokIds, batched_ques_tokIds, batched_tgt_tokIds)

ModelInput = tuple[torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor]
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=pad_collate, drop_last=True)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=pad_collate, drop_last=True)

7000
1034
[1196, 70, 44, 92]


# Self-attention

## The Masked Attention Math trick

The Triangular mask technique

In [43]:
# The Attention Mask Math trick - The Triangular mask technique
T = 300

# A triangular mask is a tensor like this: dimension (block_size, block_size)
# tensor([[1., 0., 0.,  ..., 0., 0., 0.],
#        [1., 1., 0.,  ..., 0., 0., 0.],
#        [1., 1., 1.,  ..., 0., 0., 0.],
#        ...,
#        [1., 1., 1.,  ..., 1., 0., 0.],
#        [1., 1., 1.,  ..., 1., 1., 0.],
#        [1., 1., 1.,  ..., 1., 1., 1.]])
tril_mask = torch.tril(torch.ones(T, T))

# Initial weight represents the initial *affinity* between each pair of tokens in the sequence
# (right now we initialize them all with 0s, but they can be diff numbers coming from somewhere like the encoder for example)
initial_weight = torch.zeros((T, T))

# For the purpose of decoder calculation, we want prediction of the next step to only depend on values of previous steps
# Therefore, we'll apply a *triangular mask* to the weight vector
# By the structure of the triangular mask, we can see that, after masking,
# we essentially "disable"/"mask away" future tokens since they will be multiplying with 0
# since only positions corresponding to 1s will be retained. Positions with 0s is replaced with -inf
weight = initial_weight.masked_fill(tril_mask == 0, float("-inf"))
# A softmax layer convert these *affinity scores* to a weight number between 0 and 1
weight = F.softmax(weight, dim=-1)
print(weight)
# tensor([[1.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
#        [0.5000, 0.5000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
#        [0.3333, 0.3333, 0.3333,  ..., 0.0000, 0.0000, 0.0000],
#        ...,
#        [0.0034, 0.0034, 0.0034,  ..., 0.0034, 0.0000, 0.0000],
#        [0.0033, 0.0033, 0.0033,  ..., 0.0033, 0.0033, 0.0000],
#        [0.0033, 0.0033, 0.0033,  ..., 0.0033, 0.0033, 0.0033]])
# --> See how the weight vector showcase how to calculate a prediction based on the weighted sum of previous values only
# For example:
#   2nd value = 1.00 * 1st val
#   3rd prediction = 0.5 * 1st val + 0.5 * 2nd val
#   4th prediction = 0.33 * 1st value + 0.33 * 2nd val + 0.33 * 3rd val

tensor([[1.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0034, 0.0034, 0.0034,  ..., 0.0034, 0.0000, 0.0000],
        [0.0033, 0.0033, 0.0033,  ..., 0.0033, 0.0033, 0.0000],
        [0.0033, 0.0033, 0.0033,  ..., 0.0033, 0.0033, 0.0033]])


## Single head attention block

In [44]:
# The dimension of the query, key, and value vectors/matrixes
d_model = 256

In [144]:
from math import sqrt

BLOCK_SIZE = max_len_col_name_part + max_len_tbl_name_part + max_len_ques_part

class SingleAttentionHead(nn.Module):
  def __init__(self, emb_size: int, d_model: int):
    super().__init__()
    self.emb_size = emb_size
    self.d_model = d_model
    # Each head has its own W_Q, W_K, and W_V matrixes for transform the each tok emd to its corresponding q, k, v vectors
    self.query_matrix = nn.Linear(emb_size, d_model, bias=False)
    self.key_matrix = nn.Linear(emb_size, d_model, bias=False)
    self.value_matrix = nn.Linear(emb_size, d_model, bias=False)
    # tril_mask is a static non-learned parameter, so need to use `register_buffer`
    self.register_buffer("tril_mask", torch.tril(torch.ones(BLOCK_SIZE, BLOCK_SIZE)))

  def forward(self, x: torch.Tensor, should_mask:bool=True):
    """
    Parameters:
      x: the input embedding (after summing token_emb and position_emb) --> dimension (B, T, E)
      should_mask: should this Attention block use masked attention (decoder should use mask, encoder shouldn't)
    """
    query = self.query_matrix(x) # (B, T, D) with D = d_model
    key = self.key_matrix(x) # (B, T, D)
    value = self.value_matrix(x) # (B, T, D)
    # `q @ k.T` will produce the itcanitial affinity matrix, basically how strong each query relates to each key
    # dimension: (B, T, D) @ (B, D, T) = (B, T, T)
    # Note The original "Attention is all you need paper" also scales down the affinity scores by multiplying `sqrt(d_model)`
    affinity = (query @ key.transpose(-2, -1)) * (sqrt(d_model))  # tranpose(-2, -1) avoid transposing the Batch dimension
    if should_mask:
      affinity = affinity.masked_fill(self.tril_mask == 0, float("-inf"))
    weight = F.softmax(affinity, dim=-1) # (B, T, T)
    # The output is the inner embeddings (dimension d_model) where each token's embedding have been tweaked
    # to also include information about other related tokens
    out = weight @ value # (B, T, D)
    return out

1310


# Transformer

In [148]:
from torch.nn import functional as F

src_vocab_size = len(src_vocab)
tgt_vocab_size = len(tgt_vocab)

class Transformer1(nn.Module):
  def __init__(self, emb_size: int = 300, d_model: int = 256):
    """
    Parameters:
      emb_size: the size of each word embeddings. For example: GloVe embeddings is 300, BERT is 768
    """
    super().__init__()
    self.emb_size = emb_size
    self.d_model = d_model
    self.self_attention_head = SingleAttentionHead(emb_size, d_model=d_model)
    # ENCODER COMPONENTS
    self.encoder_token_emb = nn.Embedding(src_vocab_size, emb_size)
    # Position embedding table: Convert each token's position in its block to a position embedding
    self.position_embedding_table = nn.Embedding(BLOCK_SIZE, emb_size)

    # DECODER COMPONENTS
    self.decoder_hidden_state_size = 2 * d_model
    # Embedding lookup table: Convert token_ids to that token's corresponding embeddings
    self.decoder_token_emb = nn.Embedding(tgt_vocab_size, emb_size)
    # Target language modeling head: Transform back from the embedding dimension to the tgt_vocab_size dimension
    # So that we can get the distribution and know which target token to choose
    self.tgt_lm_head = nn.Linear(self.decoder_hidden_state_size, tgt_vocab_size)
    # For the decoder, we try to replicate the RNN model to process sequences
    # decoder_hidden_state = sigmoid(W1 * context_matrix + W2 * prev_hidden_state + bias + W3 * decoder_input_tok_emb)
    # Thus we need 3 weights matrix for use in the decoder, to produce a new decoder_hidden_state
    self.decoder_context_linear = nn.Linear(d_model, self.decoder_hidden_state_size, bias=False)
    self.decoder_hiddenstate_linear = nn.Linear(self.decoder_hidden_state_size, self.decoder_hidden_state_size)
    self.decoder_token_emb_linear = nn.Linear(emb_size, self.decoder_hidden_state_size, bias=False)

  def forward(self, input_idx: torch.Tensor, target_idx: Union[torch.Tensor, None] = None):
    """
    Parameters:
      target_idx: the list of target tokens across the batches. Dimension (B, T)
    """
    x = self.encode(input_idx) # (B, T, E)
    # Average the last hidden state of the encoder as the context
    context_emb = torch.mean(x, dim=1) # (B, E)
    # Feed the <START> token as the chosen token to the entire batch
    chosen_tokens = torch.tensor(toks_encode(["<START>"] * x.size(0), "target"), device=DEVICE) # (B)
    ys: list[torch.Tensor] = []
    # Initialize the first hidden state from the last hidden state from encoder
    hidden_state = self.decoder_context_linear(context_emb) # (B, H) where H = dec_hidden_state
    for _ in range(max_len_target):
      # hidden_state: (B, H)
      # tgt_probs: (B, tgt_vocab_size)
      hidden_state, tgt_probs = self.decode(context_emb, chosen_tokens, hidden_state)
      # Greedily select the token with highest prob from the distribution
      chosen_tokens = torch.argmax(tgt_probs, dim=1) # (B)
      ys.append(tgt_probs)

    # Note that ys is collected by looping over max_len_target, so when stacked, the first dimension is max_len_target
    y = torch.stack(ys) # (T, B, C) where C = tgt_vocab_size
    assert y.shape == (max_len_target, BATCH_SIZE, tgt_vocab_size)
    if target_idx is None:
      return ys, None
    # Cross_entropy requires the "Class" dimension to be the 2nd dimension
    y = torch.permute(y, (1, 2, 0)) # --> (B, C, T)
    B, C, T = y.shape
    assert target_idx.size() == (B, T)
    # Calculate loss
    loss = F.cross_entropy(y, target_idx, ignore_index=toks_encode([PAD_TOKEN], "target")[0])
    return ys, loss

  def encode(self, input_batch: torch.Tensor):
    # Input batch is of shape (B, T) (i.e. (batch size, block_size))
    token_emb = self.encoder_token_emb(input_batch) # (B, T, E) where E=emb_size
    B, T, E = token_emb.shape
    # the position_embedding_table takes input the position of each token in the sequence (i.e. the T dimension)
    position_emb = self.position_embedding_table(torch.arange(T, device=DEVICE)) # (T, E)
    # For the bigram model, let's not do anything with these embeddings, and just return them as logits
    x = token_emb + position_emb # (B, T, E)
    # Feed this x through layers of Transformer Self-Attention blocks
    x = self.self_attention_head(x)
    return x

  def decode(self, context_emb: torch.Tensor, input_tokenIds: torch.Tensor, prev_hidden_state: torch.Tensor):
    """Decode the logis from the encoder to produce a target token"""
    # Right now let's use an RNN-like decoder
    assert context_emb.shape == (BATCH_SIZE, self.d_model)
    assert input_tokenIds.shape == (BATCH_SIZE,)
    assert prev_hidden_state.shape == (BATCH_SIZE, self.decoder_hidden_state_size), f"Got {prev_hidden_state.shape}"
    tok_emb = self.decoder_token_emb(input_tokenIds) # (B, D) where D = d_model
    assert tok_emb.shape == (BATCH_SIZE, self.emb_size)
    # For the decoder, we try to replicate the RNN model to process sequences
    # decoder_hidden_state = sigmoid(W1 * context_matrix + W2 * prev_hidden_state + W3 * decoder_input_tok_emb + bias)
    temp1 = self.decoder_context_linear(context_emb) # (B, dec_hidden_size)
    temp2 = self.decoder_hiddenstate_linear(prev_hidden_state) # (B, dec_hiden_size)
    temp3 = self.decoder_token_emb_linear(tok_emb) # (B, dec_hidden_size)
    z = temp1 + temp2 + temp3 # (B, dec_hidden_size)
    hidden_state = F.sigmoid(z) # (B, dec_hidden_size)
    assert hidden_state.shape == (BATCH_SIZE, self.decoder_hidden_state_size)

    tgt_distribution: torch.Tensor = self.tgt_lm_head(hidden_state) # (B, tgt_vocab_size)
    # Do NOT run softmax here as the Pytorch Cross Entropy Loss function expects unnormalized numbers
    # tgt_probs = F.softmax(tgt_distribution, dim=-1) # (B, tgt_vocab_size)
    return hidden_state, tgt_distribution

  @torch.no_grad()
  def generate(self, input_idx: torch.Tensor, max_generated_tokens: int = 20):
    encoder_last_hidden_state = self.encode(input_idx) # (B, T, E)
    # Average all input tokens embs across the encoder last hidden state as the context
    context_emb = torch.mean(encoder_last_hidden_state, dim=1) # (B, E)
    chosen_tokens = torch.tensor(toks_encode(["<START>"] * BATCH_SIZE, "target"), device=DEVICE) # (B)
    first_batch_predicted_tokens: list[int] = []
    # Initialize the first hidden state from the last hidden state from encoder
    hidden_state = self.decoder_context_linear(context_emb) # (B, dec_hidden_state)
    for _ in range(max_generated_tokens):
      # hidden_state: dimension (B, H)
      # tgt_probs: dimension (B, tgt_vocab_size)
      hidden_state, tgt_probs = self.decode(context_emb, chosen_tokens, prev_hidden_state=hidden_state)
      # Greedily select the token with highest prob from the distribution
      chosen_tokens = torch.argmax(tgt_probs, dim=1) # (B)
      # print(chosen_tokens)
      chosen_token = chosen_tokens[0].item() # View result of only the first batch
      first_batch_predicted_tokens.append(int(chosen_token))
    return first_batch_predicted_tokens


In [149]:
m1 = Transformer1()
m1 = m1.to(DEVICE)

m1.eval()
x_batch = next(iter(train_dataloader))

input = torch.cat((x_batch[0], x_batch[1], x_batch[2]), dim=1) # (B, block_size)

first_batch_predicted_tokens = m1.generate(input)
toks_decode(first_batch_predicted_tokens, "target")

['claims',
 'dateundergoes',
 'dateundergoes',
 'dateundergoes',
 'dateundergoes',
 'dateundergoes',
 'dateundergoes',
 'dateundergoes',
 'dateundergoes',
 'dateundergoes',
 'dateundergoes',
 'dateundergoes',
 'dateundergoes',
 'dateundergoes',
 'dateundergoes',
 'dateundergoes',
 'dateundergoes',
 'dateundergoes',
 'dateundergoes',
 'dateundergoes']

In [152]:
# Train the network
# Create an optimizer
m1.train()
optimizer = torch.optim.AdamW(m1.parameters(), lr=5e-6)
train_dataloader_iter = iter(train_dataloader)

for epoch in range(300):
  try:
    batch = next(train_dataloader_iter)
  except StopIteration:
    # Reset the dataloader
    train_dataloader_iter = iter(train_dataloader)
    batch = next(train_dataloader_iter)
  input = torch.cat((batch[0], batch[1], batch[2]), dim=1) # (B, block_size)
  target = batch[3] # (B, max_len_target)
  ys, loss = m1(input, target)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()
  print(epoch, loss)

0 tensor(8.2893, device='mps:0', grad_fn=<NllLoss2DBackward0>)
1 tensor(8.3051, device='mps:0', grad_fn=<NllLoss2DBackward0>)
2 tensor(8.2695, device='mps:0', grad_fn=<NllLoss2DBackward0>)
3 tensor(8.3076, device='mps:0', grad_fn=<NllLoss2DBackward0>)
4 tensor(8.3521, device='mps:0', grad_fn=<NllLoss2DBackward0>)
5 tensor(8.2585, device='mps:0', grad_fn=<NllLoss2DBackward0>)
6 tensor(8.2283, device='mps:0', grad_fn=<NllLoss2DBackward0>)
7 tensor(8.2716, device='mps:0', grad_fn=<NllLoss2DBackward0>)
8 tensor(8.3237, device='mps:0', grad_fn=<NllLoss2DBackward0>)
9 tensor(8.2602, device='mps:0', grad_fn=<NllLoss2DBackward0>)
10 tensor(8.2499, device='mps:0', grad_fn=<NllLoss2DBackward0>)
11 tensor(8.2509, device='mps:0', grad_fn=<NllLoss2DBackward0>)
12 tensor(8.2351, device='mps:0', grad_fn=<NllLoss2DBackward0>)
13 tensor(8.2911, device='mps:0', grad_fn=<NllLoss2DBackward0>)
14 tensor(8.2242, device='mps:0', grad_fn=<NllLoss2DBackward0>)
15 tensor(8.1951, device='mps:0', grad_fn=<NllLoss

In [153]:
# After training
# Save the model
torch.save(m1.state_dict(), "transform1.pt")

In [154]:
# Inference
m1_trained = Transformer1()
m1_trained.load_state_dict(torch.load("transform1.pt"))
m1_trained = m1_trained.to(DEVICE)
m1_trained.eval()

x_batch = next(iter(train_dataloader))

input = torch.cat((x_batch[0], x_batch[1], x_batch[2]), dim=1) # (B, block_size)
target = x_batch[3].tolist()[0]
print(target)
print(toks_decode(target, "target"))

y_batch = m1_trained.generate(input, max_generated_tokens=max_len_target)
print(toks_decode(input.tolist()[0], "source"))
print(y_batch)
print(toks_decode(y_batch, "target"))

[3086, 727, 3000, 4693, 3256, 2966, 2746, 1216, 1432, 2356, 5396, 3178, 4143, 4143, 4143, 4143, 4143, 4143, 4143, 4143, 4143, 4143, 4143, 4143, 4143, 4143, 4143, 4143, 4143, 4143, 4143, 4143, 4143, 4143, 4143, 4143, 4143, 4143, 4143, 4143, 4143, 4143, 4143, 4143, 4143, 4143, 4143, 4143, 4143, 4143, 4143, 4143, 4143, 4143, 3185, 3185, 3185, 3185, 3185, 3185, 3185, 3185, 3185, 3185, 3185, 3185, 3185, 3185, 3185, 3185, 3185, 3185, 3185, 3185, 3185, 3185, 3185, 3185, 3185, 3185, 3185, 3185, 3185, 3185, 3185, 3185, 3185, 3185, 3185, 3185, 3185, 3185]
['<START>', 'SELECT', 'StuID', 'FROM', 'Student', 'WHERE', 'city_code', '=', '``', 'CHI', "''", '<END>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '